# Keylevels detection

### Import Library

In [51]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [52]:
import os
from pathlib import Path
notebook_path = os.getcwd()
current_dir = Path(notebook_path)
csv_file = str(current_dir) + '/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [53]:
data = dataset.copy()

In [54]:
data = data[data.index > '2020-11-01 00:00:00']

In [55]:
data

,Open,High,Low,Close,Volume
Date,,,,,
2020-11-02 09:00:00,900.1,900.2,899.3,900.1,1910
2020-11-02 09:05:00,900.2,900.2,898.7,899.4,1670
2020-11-02 09:10:00,899.5,900.0,899.0,899.5,1329
2020-11-02 09:15:00,899.4,899.5,898.2,898.6,1722
2020-11-02 09:20:00,898.5,898.6,896.5,898.2,2939
...,...,...,...,...,...
2025-02-14 14:15:00,1343.0,1343.0,1340.3,1341.3,7141
2025-02-14 14:20:00,1340.9,1341.9,1340.5,1341.4,4593
2025-02-14 14:25:00,1341.1,1342.5,1340.7,1342.5,4207


In [56]:
back_bar = 20
next_bar = back_bar -1
data['max_prev'] = data['High'].rolling(back_bar).max()
data['max_next'] = data['High'].shift(-next_bar).rolling(back_bar).max()
data['min_prev'] = data['Low'].rolling(back_bar).min()
data['min_next'] = data['Low'].shift(-next_bar).rolling(back_bar).min()

In [57]:
data.dropna(inplace=True)
data['is_max_last20'] = data.apply(lambda r: 1 if r['High'] == r['max_prev'] else 0, axis=1)
data['is_min_last20'] = data.apply(lambda r: 1 if r['Low'] == r['min_prev'] else 0, axis=1)
data['is_peak'] = data.apply(lambda r: 1 if (r['is_max_last20'] == 1 and r['High'] == r['max_next']) else 0, axis=1)
data['is_valley'] = data.apply(lambda r: 1 if (r['is_min_last20'] == 1 and r['Low'] == r['min_next']) else 0, axis=1)

In [58]:
data[data.is_max_last20 == 1]

,Open,High,Low,Close,Volume,max_prev,max_next,min_prev,min_next,is_max_last20,is_min_last20,is_peak,is_valley
Date,,,,,,,,,,,,,
2020-11-02 13:45:00,897.7,900.4,897.2,900.4,2889,900.4,909.4,895.4,896.6,1,0,0,0
2020-11-02 13:50:00,900.4,903.0,900.4,902.5,7061,903.0,909.4,895.4,896.6,1,0,0,0
2020-11-02 14:20:00,899.0,905.5,899.0,904.5,7646,905.5,909.4,896.3,899.0,1,0,0,0
2020-11-02 14:25:00,904.9,907.0,904.5,907.0,5873,907.0,909.4,896.3,904.0,1,0,0,0
2020-11-02 14:30:00,906.7,907.0,906.7,907.0,253,907.0,909.4,896.3,904.0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-13 13:35:00,1337.9,1338.3,1336.6,1336.9,4509,1338.3,1349.0,1327.4,1333.4,1,0,0,0
2025-02-13 13:45:00,1337.2,1338.5,1336.5,1336.9,4460,1338.5,1349.0,1329.4,1333.4,1,0,0,0
2025-02-14 09:00:00,1345.2,1348.9,1345.2,1347.6,11413,1348.9,1349.0,1333.0,1344.7,1,0,0,0


In [59]:
data[data.is_peak == 1]

,Open,High,Low,Close,Volume,max_prev,max_next,min_prev,min_next,is_max_last20,is_min_last20,is_peak,is_valley
Date,,,,,,,,,,,,,
2020-11-03 11:00:00,908.7,910.8,908.6,910.5,2674,910.8,910.8,906.8,903.7,1,0,1,0
2020-11-04 14:00:00,916.6,918.1,916.4,916.7,5942,918.1,918.1,911.8,910.2,1,0,1,0
2020-11-05 13:00:00,918.5,919.9,918.5,919.1,2509,919.9,919.9,914.5,910.0,1,0,1,0
2020-11-09 11:10:00,919.4,920.4,919.4,919.6,2289,920.4,920.4,915.5,916.0,1,0,1,0
2020-11-10 09:00:00,935.0,935.3,933.6,933.9,3434,935.3,935.3,916.0,928.1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-07 13:40:00,1343.9,1347.4,1343.8,1345.3,14175,1347.4,1347.4,1337.6,1330.0,1,0,1,0
2025-02-10 13:35:00,1335.0,1337.6,1334.9,1337.0,4192,1337.6,1337.6,1331.5,1330.0,1,0,1,0
2025-02-12 10:05:00,1340.6,1342.0,1340.5,1340.6,5056,1342.0,1342.0,1328.9,1338.6,1,0,1,0


## Classification by current price & next price

In [60]:
# Entry Long-Short late 1 bar(5 minutes) but it's not problem

In [61]:
data['next_Close'] = data['Close'].shift(-1)
data['next_Open'] = data['Open'].shift(-1)
data['next_High'] = data['High'].shift(-1)
data['next_Low'] = data['Low'].shift(-1)
data['RSI'] = ta.rsi(data["Close"], length=20)

In [62]:
ai_pdata = data[data.is_max_last20 == 1] # data for classifying peak or not

In [63]:
from sklearn.model_selection import train_test_split
# Define Features and Target Variable
features = ["Open", "Close", "Low", "High", "next_Open", "next_Close", "next_Low", "next_High", "RSI"]
X = ai_pdata[features]
y = ai_pdata["is_peak"]

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [64]:
#Fix Imbalanced Classes
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy="auto")
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

### Train & test by RandomForestClassifier

In [65]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [66]:
# Train Random Forest Model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_resampled, y_resampled)

RandomForestClassifier(random_state=42)

In [67]:
# Predictions
y_pred = model.predict(X_test)

In [68]:
# Evaluate Performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.7674750356633381
              precision    recall  f1-score   support

           0       0.89      0.84      0.86      1213
           1       0.24      0.33      0.28       189

    accuracy                           0.77      1402
   macro avg       0.56      0.58      0.57      1402
weighted avg       0.80      0.77      0.78      1402



In [69]:
probability = model.predict_proba(X_test)
probability

array([[0.98, 0.02],
       [0.47, 0.53],
       [0.86, 0.14],
       ...,
       [0.15, 0.85],
       [0.79, 0.21],
       [0.36, 0.64]])

In [70]:
result_data = X_test.copy()
result_data['is_peak'] = y_test
# result_data['predict'] = y_pred
result_data = result_data.assign(predict=y_pred)
result_data['prob_0'] = probability[:,0] 
result_data['prob_1'] = probability[:,1]
result_data

,Open,Close,Low,High,next_Open,next_Close,next_Low,next_High,RSI,is_peak,predict,prob_0,prob_1
Date,,,,,,,,,,,,,
2023-11-15 10:15:00,1149.3,1149.3,1148.6,1150.6,1149.3,1148.7,1148.7,1149.8,79.736771,1,0,0.98,0.02
2022-11-28 09:00:00,975.0,974.0,974.0,979.0,974.0,974.0,972.4,975.4,74.568877,0,1,0.47,0.53
2020-12-09 13:00:00,1002.6,1003.8,1002.6,1004.5,1003.8,1003.8,1003.7,1004.2,70.280061,0,0,0.86,0.14
2022-08-29 14:00:00,1273.5,1274.0,1273.5,1275.2,1274.0,1273.0,1271.5,1274.0,52.459487,0,0,0.57,0.43
2024-04-22 13:55:00,1213.3,1213.4,1212.2,1214.7,1213.1,1213.0,1212.8,1215.3,60.985064,0,0,0.52,0.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-02 13:15:00,1043.9,1042.8,1042.0,1046.3,1042.8,1042.3,1042.0,1044.1,61.689043,0,0,0.61,0.39
2023-04-18 13:50:00,1062.0,1063.0,1062.0,1063.4,1063.0,1063.0,1062.9,1063.8,54.252147,0,0,0.95,0.05
2025-01-17 11:15:00,1309.7,1310.6,1309.7,1311.4,1310.6,1310.1,1309.7,1310.9,61.597289,0,1,0.15,0.85


In [71]:
len(result_data[result_data.is_peak == result_data.predict]) / len(result_data)

0.7674750356633381

In [72]:
# from sklearn.model_selection import GridSearchCV

# params = {
#     "n_estimators": [100, 150, 200],
#     "max_depth": [5, 10, 20],
#     "min_samples_split": [2, 5, 10],
#     "min_samples_leaf": [1, 2, 5]
# }

# grid = GridSearchCV(RandomForestClassifier(), params, cv=5, scoring="accuracy")
# grid.fit(X_resampled, y_resampled)
# best_model = grid.best_estimator_
# print("Best Params:", grid.best_params_)
# ## Best Params: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150}

In [73]:
# # Predictions
# y2_pred = best_model.predict(X_test)
# # Evaluate Performance
# print("Accuracy:", accuracy_score(y_test, y2_pred))
# print(classification_report(y_test, y2_pred))

### Train & test by Xgboost

In [74]:
import xgboost as xgb

In [75]:
model = xgb.XGBClassifier(n_estimators=300, learning_rate=0.05, max_depth=6)
model.fit(X_resampled, y_resampled)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [76]:
y_pred = model.predict(X_test)
print("New Accuracy:", accuracy_score(y_test, y_pred))

New Accuracy: 0.6982881597717546
